In [27]:
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/Statistics with Python/Sample Data/titanic_train.csv')
test = pd.read_csv('/content/drive/MyDrive/Statistics with Python/Sample Data/titanic_test.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
train[['Pclass', 'Survived']].groupby(['Pclass']).mean().sort_values(by='Survived', ascending=False)

,Survived
Pclass,
1,0.629630
2,0.472826
3,0.242363


In [ ]:
train[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Sex,Survived
0,female,0.742038
1,male,0.188908


In [28]:
train = train.drop(['PassengerId','Name','Ticket','Cabin'], axis=1)
train.shape

train['Sex'] = train['Sex'].map({'female':1,'male':0})

In [29]:
import numpy as np
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.NaN, strategy='mean')
impMode = SimpleImputer(missing_values=np.NaN, strategy='most_frequent')
train['Age'] = imp.fit_transform(train['Age'].values.reshape(-1,1))[:,0]
train['Embarked'] = impMode.fit_transform(train['Embarked'].values.reshape(-1,1))[:,0]
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
dtypes: float64(2), int64(5), object(1)
memory usage: 55.8+ KB


In [30]:
train['AgeBand'] = pd.cut(train['Age'], 5)
train[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

,AgeBand,Survived
0,"(0.34, 16.336]",0.550000
1,"(16.336, 32.252]",0.344168
2,"(32.252, 48.168]",0.404255
3,"(48.168, 64.084]",0.434783
4,"(64.084, 80.0]",0.090909


In [35]:
train.loc[train['Age']<=16, 'AgeGroup'] = 0
train.loc[(train['Age']>16) & (train['Age']<=32), 'AgeGroup'] = 1
train.loc[(train['Age']>32) & (train['Age']<=48), 'AgeGroup'] = 2
train.loc[(train['Age']>48) & (train['Age']<=64), 'AgeGroup'] = 3
train.loc[train['Age']>64, 'AgeGroup'] = 4

In [36]:
train['FareBand'] = pd.qcut(train['Fare'], 4)
train[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

,FareBand,Survived
0,"(-0.001, 7.91]",0.197309
1,"(7.91, 14.454]",0.303571
2,"(14.454, 31.0]",0.454955
3,"(31.0, 512.329]",0.581081


In [37]:
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
train['IsAlone'] = 0
train.loc[train['FamilySize'] == 1, 'IsAlone'] = 1
train[['IsAlone','Survived']].groupby(['IsAlone'],as_index=False).mean()

,IsAlone,Survived
0,0,0.505650
1,1,0.303538


In [38]:
train['Age*Class'] = train['AgeGroup'] * train['Pclass']
train.loc[:, ['Age*Class','AgeGroup','Pclass']].head()

,Age*Class,AgeGroup,Pclass
0,3.0,1.0,3
1,2.0,2.0,1
2,3.0,1.0,3
3,2.0,2.0,1
4,6.0,2.0,3


In [39]:
dummy = pd.get_dummies(train['Embarked'])
train = train.join(dummy)

In [40]:
X_train = train[['Pclass','Sex','AgeGroup','Fare','Q','S','IsAlone','Age*Class']]
y_train = train['Survived']

In [42]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(X_train,y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [43]:
logreg.intercept_.round(3)

array([1.595])

In [47]:
logreg.coef_.round(3)

array([[-0.93 ,  2.498, -0.219,  0.   , -0.051, -0.443,  0.01 , -0.142]])

In [45]:
from sklearn.metrics import classification_report, confusion_matrix

confusion_matrix(y_train,logreg.predict(X_train))

array([[467,  82],
       [101, 241]])

In [46]:
acc_log = round(logreg.score(X_train, y_train)*100, 2)
acc_log

79.46

In [49]:
from sklearn.svm import SVC

svc = SVC()
svc.fit(X_train,y_train)
acc_svc = round(svc.score(X_train, y_train)*100, 2)
acc_svc

67.45

In [50]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train,y_train)
acc_knn = round(knn.score(X_train,y_train)*100, 2)
acc_knn

85.3

In [52]:
from sklearn.naive_bayes import GaussianNB

gaussian = GaussianNB()
gaussian.fit(X_train, y_train)
acc_gaussian = round(gaussian.score(X_train, y_train)*100, 2)
acc_gaussian

76.99

In [54]:
from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
acc_decision_tree = round(decision_tree.score(X_train, y_train)*100, 2)
acc_decision_tree

93.49

In [57]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)
random_forest.score(X_train, y_train)
acc_random_forest = round(random_forest.score(X_train, y_train)*100, 2)
acc_random_forest

93.49

In [59]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(10,10,2),random_state=1)
mlp.fit(X_train, y_train)
mlp.score(X_train, y_train)
acc_mlp = round(mlp.score(X_train, y_train)*100, 2)
acc_mlp

80.25

In [62]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines','K-Nearest Neighbors','Logistic Regression','Random Forest','Naive Bayes','Neural Network','Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log, acc_random_forest, acc_gaussian, acc_mlp, acc_decision_tree]
})
models.sort_values(by='Score',ascending=False)

,Model,Score
3,Random Forest,93.49
6,Decision Tree,93.49
1,K-Nearest Neighbors,85.30
5,Neural Network,80.25
2,Logistic Regression,79.46
4,Naive Bayes,76.99
0,Support Vector Machines,67.45


In [63]:
test['Sex'] = test['Sex'].map({'female':1, 'male':0})
test['Age'] = imp.fit_transform(test['Age'].values.reshape(-1,1))[:,0]
test['Fare'] = imp.fit_transform(test['Fare'].values.reshape(-1,1))[:,0]
test['Embarked'] = impMode.fit_transform(test['Embarked'].values.reshape(-1,1))[:,0]

In [64]:
test.loc[test['Age']<=16,'AgeGroup'] = 0
test.loc[(test['Age']>16) & (test['Age']<=32), 'AgeGroup'] = 1
test.loc[(test['Age']>32) & (test['Age']<=48), 'AgeGroup'] = 2
test.loc[(test['Age']>48) & (test['Age']<=64), 'AgeGroup'] = 3
test.loc[test['Age']>64, 'AgeGroup'] = 4

In [65]:
test['FamilySize'] = test['SibSp'] + test['Parch'] + 1
test['IsAlone'] = 0
test.loc[train['FamilySize'] == 1, 'IsAlone'] = 1

In [66]:
test['Age*Class'] = test['AgeGroup'] * test['Pclass']

In [67]:
dummy = pd.get_dummies(test['Embarked'])
test = test.join(dummy)

In [70]:
X_test = test[['Pclass','Sex','AgeGroup','Fare','Q','S','IsAlone','Age*Class']]

In [71]:
y_pred = random_forest.predict(X_test)
submit = pd.DataFrame({'PassengerId': test['PassengerId'],
                       'Survived': y_pred})
submit

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [72]:
submit.to_csv('/content/drive/MyDrive/Statistics with Python/submission.csv', index=False)